In [11]:
import numpy as np
import pygrib

## combine everything

In [12]:
gfs_path = '/Users/'

dates = ['20230206']
f_time = ['00','06' ,'12', '18']
lat_range = [25.1, 50]
lon_range = [131, 231]



In [13]:
for f_t in f_time:
    
    for date in dates:
        h = 0
        lwindspeed = []
        lwinddirection = []
        lshww = []
        lshsw = []
        lpww = []
        lpsw = []
        ldww = []
        ldsw = []
        lwu = []
        lwv = []
        while h <= 384:
            hn = '00'+str(h) if h <= 9 else '0' + str(h) if h<100  else str(h)

            wave = pygrib.open(gfs_path+date+'/'+f_t+'/'+'gfswave.t'+f_t+'z.global.0p25.f'+hn+'.grib2')
            wave.seek(0)
            Twindspeed, _, _ = wave[1].data(lat1=lat_range[0],lat2=lat_range[1],lon1=lon_range[0],lon2=lon_range[1])
            Twinddirection, _, _ = wave[2].data(lat1=lat_range[0],lat2=lat_range[1],lon1=lon_range[0],lon2=lon_range[1])
            Tshww, _, _ = wave[8].data(lat1=lat_range[0],lat2=lat_range[1],lon1=lon_range[0],lon2=lon_range[1])
            Tshsw, _, _ = wave[9].data(lat1=lat_range[0],lat2=lat_range[1],lon1=lon_range[0],lon2=lon_range[1])
            Tpww, _, _ = wave[12].data(lat1=lat_range[0],lat2=lat_range[1],lon1=lon_range[0],lon2=lon_range[1])
            Tpsw, _, _ = wave[13].data(lat1=lat_range[0],lat2=lat_range[1],lon1=lon_range[0],lon2=lon_range[1])
            Tdww, _, _ = wave[16].data(lat1=lat_range[0],lat2=lat_range[1],lon1=lon_range[0],lon2=lon_range[1])
            Tdsw, _, _ = wave[17].data(lat1=lat_range[0],lat2=lat_range[1],lon1=lon_range[0],lon2=lon_range[1])
            #Twind_u, _, _ = wave[3].data(lat1=lat_range[0],lat2=lat_range[1],lon1=lon_range[0],lon2=lon_range[1])
            #Twind_v, _, _ = wave[4].data(lat1=lat_range[0],lat2=lat_range[1],lon1=lon_range[0],lon2=lon_range[1])
    
            lwindspeed.append(Twindspeed.data)
            lwinddirection.append(Twinddirection.data)
            lshww.append(Tshww.data)
            lshsw.append(Tshsw.data)
            lpww.append(Tpww.data)
            lpsw.append(Tpsw.data)
            ldww.append(Tdww.data)
            ldsw.append(Tdsw.data)
            #lwu.append(Twind_u.data)
            #lwv.append(Twind_v.data) 
    
            h = h+1 if h<120 else h+3
        windspeed = np.stack(lwindspeed,axis = 2)
        winddirection = np.stack(lwinddirection,axis = 2)
        shww = np.stack(lshww,axis = 2)
        shsw = np.stack(lshsw, axis = 2)
        pww = np.stack(lpww, axis = 2)
        psw = np.stack(lpsw, axis = 2)
        dww = np.stack(ldww, axis = 2)
        dsw = np.stack(ldsw, axis = 2)
        #wind_u = np.stack(lwu,axis = 2)
        #wind_v = np.stack(lwv,axis = 2)
        all_weather = np.stack((windspeed, winddirection, shww,pww,dww,shsw,psw,dsw), axis = 3)
        np.save('25gfs'+date+'_'+f_t, all_weather.data)
        #np.save('25gfs_wind_u'+date+'_'+f_t, wind_u.data)
        #np.save('25gfs_wind_v'+date+'_'+f_t, wind_v.data)
        
print('Finished normally')    

Finished normally


In [14]:
def nan_interpolate(arr_3d):
    '''This function is used to interpolate the NaN values'''
    result=np.zeros_like(arr_3d)
    for i in range(arr_3d.shape[0]):
        for j in range(arr_3d.shape[1]):
            arr=arr_3d[i,j,:]
            # If all elements are nan then cannot conduct linear interpolation.
            if np.sum(np.isnan(arr))==arr.shape[0]:
                result[i,j,:]=0
            else:
                # If the first elemet is nan, then assign the value of its right nearest neighbor to it.
                if np.isnan(arr[0]):
                    arr[0]=arr[~np.isnan(arr)][0]
                # If the last element is nan, then assign the value of its left nearest neighbor to it.
                if np.isnan(arr[-1]):
                    arr[-1]=arr[~np.isnan(arr)][-1]
                # If the element is in the middle and its value is nan, do linear interpolation using neighbor values.
                for k in range(arr.shape[0]):
                    if np.isnan(arr[k]):
                        x=k
                        x1=x-1
                        x2=x+1
                        # Find left neighbor whose value is not nan.
                        while x1>=0:
                            if np.isnan(arr[x1]):
                                x1=x1-1
                            else:
                                y1=arr[x1]
                                break
                        # Find right neighbor whose value is not nan.
                        while x2<arr.shape[0]:
                            if np.isnan(arr[x2]):
                                x2=x2+1
                            else:
                                y2=arr[x2]
                                break
                        # Calculate the slope and intercept determined by the left and right neighbors.
                        t1 = timeframe2hour(x1)
                        diff=(y2-y1)/(timeframe2hour(x2)-t1)                        
                        y=y1+diff*(np.abs(t1-timeframe2hour(x)))
                        arr[x]=y
                result[i,j,:]=arr
    return result

In [15]:
def timeframe2hour(tf):
    '''The number for the hours is from 0 to 209, but the real time frame is from 0 to 384. 
    From hour 120 onwards, every three hours, there is one frame of data. 
    So that the time frame has to be converted to real hour for interpolation of the weather attributes.'''
    if tf > 120:
        tf = (tf-120)*3 + 120
    return tf

In [16]:
s = 5
date = '20230206'
for tf in f_time:
    all_weather = np.load('25gfs'+date+'_'+tf+'.npy')[:, 100:, :, :]
    all_weather = all_weather[:,::s,:,:]
    all_weather[all_weather==9999]=np.nan   # to replace the invalid 9999 into np.NaN
    tws = []
    for w in range(all_weather.shape[3]):
        tw = nan_interpolate(all_weather[:,:,:,w])
        tws.append(tw)
    all_weather = np.stack((tws), axis = 3)
    np.save('nonan_'+date+tf, all_weather)